### Import Necessary Dependencies

In [1]:
import fitz
from PIL import Image
import pytesseract
import io
import re
import os
import requests
import pickle
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
from langchain.memory import ConversationBufferMemory
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

D:\10-Minute-School\Multilingual-Literature-Rag\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### OCR for data extraction and Clean text function

In [2]:

def extract_text_from_pdf(pdf_path: str) -> str:
    custom_config = r'--oem 3 --psm 6 -l ben+eng'
    doc = fitz.open(pdf_path)
    all_text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap(dpi=300)
        img = Image.open(io.BytesIO(pix.tobytes("png")))
        text = pytesseract.image_to_string(img, config=custom_config)
        all_text += text + "\n\n"
    return all_text

def clean_bangla_ocr_text(raw_text: str) -> str:
    text = re.sub(r'\n{3,}', '\n\n', raw_text)
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r' *\n *', '\n', text)
    text = re.sub(r'(\S)-\n(\S)', r'\1\2', text)
    text = re.sub(r'(?<![\।।।:])\n(?![\n•])', ' ', text)
    text = re.sub(r'\b(ক|খ|গ|ঘ|ঙ|চ|ছ|জ|ঝ|ঞ|ট|ঠ|ড|ঢ|ণ|ত|থ|দ|ধ|ন|প|ফ|ব|ভ|ম|য|র|ল|শ|ষ|স|হ|ড়|ঢ়|য়)\)', '', text)
    text = re.sub(r'[^\u0980-\u09FFA-Za-z0-9\s.,:;!?()\[\]\"\'\-–—\n]', '', text)
    return text.strip()

### Chunking

In [3]:
def chunk_text(text, chunk_size=300, overlap=80):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    return splitter.split_text(text)

### Embedding the text store FAISS database and Retriev from FAISS database

In [4]:
def embed_and_store(chunks, model_name="intfloat/multilingual-e5-large", faiss_dir="../faiss_index"):
    os.makedirs(faiss_dir, exist_ok=True)
    index_path = os.path.join(faiss_dir, "index.faiss")
    chunks_path = os.path.join(faiss_dir, "chunks.pkl")
    model = SentenceTransformer(model_name)
    if os.path.exists(index_path) and os.path.exists(chunks_path):
        index = faiss.read_index(index_path)
        with open(chunks_path, "rb") as f:
            chunks = pickle.load(f)
    else:
        embeddings = model.encode(chunks, show_progress_bar=True)
        dimension = embeddings.shape[1]
        index = faiss.IndexFlatL2(dimension)
        index.add(np.array(embeddings))
        faiss.write_index(index, index_path)
        with open(chunks_path, "wb") as f:
            pickle.dump(chunks, f)
    return index, chunks, model

def get_relevant_chunks(query, chunks, embedder, faiss_index, top_k=5):
    query_embedding = embedder.encode([query])
    D, I = faiss_index.search(query_embedding, top_k)
    return [chunks[i] for i in I[0]]

### Answer Generation using Ollama mistral model

In [5]:

def generate_answer_ollama(prompt, model="mistral", host="http://localhost:11434"):
    endpoint = f"{host}/api/generate"
    headers = {"Content-Type": "application/json"}
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "options": {
            "temperature": 0.2,  # Lower temperature for more focused answers
            "top_p": 0.85,
            "repeat_penalty": 1.1,
            "num_ctx": 3072  # Increased context window
        }
    }
    response = requests.post(endpoint, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()["response"].strip()
    else:
        return f"Error: {response.status_code} - {response.text}"

### Few Short Prompt Template

In [6]:
def format_prompt(context, question):
    return f"""আপনি একজন বুদ্ধিমান সহকারী। নিচের প্রসঙ্গ পড়ে প্রশ্নের উত্তর দিন।
                প্রসঙ্গ:
                {context}
                
                প্রশ্ন:
                {question}
                
                উত্তর:"""

### Memory for recent inputs in the chat sequence and Evaluation matrix

In [8]:
memory = ConversationBufferMemory(return_messages=True)

def evaluate_rag_retrieval(query, context_chunks, generated_answer, embedder):
    context_text = " ".join(context_chunks)
    query_embedding = embedder.encode([query])
    context_embedding = embedder.encode([context_text])
    answer_embedding = embedder.encode([generated_answer])
    relevance = cosine_similarity(query_embedding, context_embedding)[0][0]
    groundedness = cosine_similarity(answer_embedding, context_embedding)[0][0]
    return relevance, groundedness

C:\Users\Md. Reyad Hossain\AppData\Local\Temp\ipykernel_16016\2486021696.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


In [9]:
text = extract_text_from_pdf("../data/HSC26-Bangla1st-Paper.pdf")
print("All text after extracting from pdf : ",text[:1000])

All text after extracting from pdf :  [16০১০
HSC 26
- = >)
অনলাইন ব্যাচ
বাংলা
১ম পত্র
আলোচ্য বিষয়
অপরিচিতা
অনলাইন ব্যাচ সম্পর্কিত যেকোনো জিজ্ঞাসায়,
কল করো 16910


=o 1০১৬
/ নিম্নবিত্ত ব্যক্তির হঠাৎ বিত্তশালী হয়ে ওঠার ফলে সমাজে পরিচয় সংকট সম্পর্কে ধারণা লাভ করবে।
/ তৎকালীন সমাজ-সভ্যতা ও মানবতার অবমাননা সম্পর্কে জানতে পারবে।
/ তৎকালীন সমাজের পণপ্রথার কুপ্রভাব সম্পর্কে জানতে পারবে।
/ তৎকালে সমাজে ভদ্রলোকের স্বভাববৈশিষ্ট্য সম্পর্কে জ্ঞানলাভ করবে।
/ নারী কোমল ঠিক, কিন্তু দুর্বল নয়- কল্যাণীর জীবনচরিত দ্বারা প্রতিষ্ঠিত এই সত্য অনুধাবন করতে
পারবে।
/ মানুষ আশা নিয়ে বেঁচে থাকে- অনুপমের Wis মানবজীবনের এই চিরন্তন সত্যদর্শন সম্পর্কে
জ্ঞানলাভ করবে।
১। অনুপমের বাবা কী করে জীবিকা নির্বাহ করতেন?
ক) ডাক্তারি খ) ওকালতি গ) মাস্টারি ঘ) ব্যবসা
২। মামাকে ভাগ্য দেবতার প্রধান এজেন্ট বলার কারণ, তার-
ক) প্রতিপত্তি খ) প্রভাব গ) বিচক্ষণতা ঘ) Po বুদ্ধি
নিচের অনুচ্ছেদটি পড়ে ৩ ও ৪ সংখ্যক প্রশ্নের উত্তর দাও।
frosts দীপুর চাচাই ছিলেন পরিবারের SST দীপু শিক্ষিত হলেও তার সিদ্ধান্ত নেওয়ার ক্ষমতা ছিল না। চাচা
তার বি

In [10]:
clean_text = clean_bangla_ocr_text(text)
print("After clean up the all text : ",clean_text[:1000])

After clean up the all text :  [16০১০ HSC 26 -  ) অনলাইন ব্যাচ বাংলা ১ম পত্র আলোচ্য বিষয় অপরিচিতা অনলাইন ব্যাচ সম্পর্কিত যেকোনো জিজ্ঞাসায়, কল করো 16910
 o 1০১৬  নিম্নবিত্ত ব্যক্তির হঠাৎ বিত্তশালী হয়ে ওঠার ফলে সমাজে পরিচয় সংকট সম্পর্কে ধারণা লাভ করবে
 তৎকালীন সমাজ-সভ্যতা ও মানবতার অবমাননা সম্পর্কে জানতে পারবে
 তৎকালীন সমাজের পণপ্রথার কুপ্রভাব সম্পর্কে জানতে পারবে
 তৎকালে সমাজে ভদ্রলোকের স্বভাববৈশিষ্ট্য সম্পর্কে জ্ঞানলাভ করবে
 নারী কোমল ঠিক, কিন্তু দুর্বল নয়- কল্যাণীর জীবনচরিত দ্বারা প্রতিষ্ঠিত এই সত্য অনুধাবন করতে পারবে
 মানুষ আশা নিয়ে বেঁচে থাকে- অনুপমের Wis মানবজীবনের এই চিরন্তন সত্যদর্শন সম্পর্কে জ্ঞানলাভ করবে
১ অনুপমের বাবা কী করে জীবিকা নির্বাহ করতেন?  ডাক্তারি  ওকালতি  মাস্টারি  ব্যবসা ২ মামাকে ভাগ্য দেবতার প্রধান এজেন্ট বলার কারণ, তারক) প্রতিপত্তি  প্রভাব  বিচক্ষণতা  Po বুদ্ধি নিচের অনুচ্ছেদটি পড়ে ৩ ও ৪ সংখ্যক প্রশ্নের উত্তর দাও
frosts দীপুর চাচাই ছিলেন পরিবারের SST দীপু শিক্ষিত হলেও তার সিদ্ধান্ত নেওয়ার ক্ষমতা ছিল না চাচা তার বিয়ের উদ্যোগ নিলেও যৌতুক নিয়ে বাড়াবাড়ি কর

In [11]:
chunks = chunk_text(clean_text) # Split the cleaned text into overlapping chunks for semantic search
index, chunks, embedder = embed_and_store(chunks) # Generate embeddings, build or load FAISS index, and return the embedder model

Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [01:42<00:00,  7.91s/it]


### Sample questions for testing the RAG pipeline

In [12]:

sample_questions = [
    "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?",
    "অনুপম কাকে ভাগ্যদেবতা বলেছেন?",
    "বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?",
    "অনুপমের জীবনে মামার ভূমিকা কী ছিল?",
    "'অপরিচিতা' গল্পে বিয়ে ভেঙে যাওয়ার মূল কারণ কী ছিল?",
    "গল্পে 'ফল্গু নদী' উপমার মাধ্যমে কী বোঝানো হয়েছে?",
    "'অপরিচিতা' গল্পে অনুপম নিজেকে কীভাবে বর্ণনা করেছে?",
    "'অপরিচিতা' গল্পে যে কনে প্রত্যাখ্যান করেছিল, তার প্রতি অনুপমের দৃষ্টিভঙ্গি কেমন ছিল?",
    "যদি যতা, আমি একবার বিবাহ আসরে যাই,  এই বাক্যের পেছনে অনুপমের মানসিক অবস্থা কী ছিল?",
    "গল্পে মামার 'বর পছন্দের মানদণ্ড' কী ছিল?",
    "অনুপমকে 'ভদ্রলোক' বলা হলেও তার মধ্যে কী কী দুর্বলতা ছিল?",
    "'অপরিচিতা' গল্পে বিয়ের আসরে কী ধরনের অপমানজনক ঘটনা ঘটেছিল?",
    "গল্পে 'যার মুখ সুরূপ, সে যদি মনের দিক দিয়ে অযোগ্য হয় তবে?' – এই ধরনের বক্তব্য কী বোঝায়?",
    "আলোচনা করো: ভাষার সৌন্দর্য কোথায়?",
    "লালনের ভাবনা কেন গুরুত্বপূর্ণ?",
    "হাসন রাজা কীভাবে সমাজকে দেখেছেন?",
    "সাহিত্য ও বাস্তবতার সম্পর্ক কী?",
    "What role did Upanupom's maternal uncle (mama) play in his marriage proposal?",
    "In the story 'Aparichita', why did the marriage not happen eventually?",
    "What is the symbolic meaning of the 'Phalgu River' metaphor in the story?",
    "How did Anupom perceive himself regarding wealth and societal status?",
    "What social critique does Tagore offer through the rejection of the marriage in 'Aparichita'?"
]
# Loop through questions
for i, question in enumerate(sample_questions, 1):
    relevant_chunks = get_relevant_chunks(question, chunks, embedder, index)
    context = "\n\n".join(relevant_chunks)
    
    # Memory chat history
    chat_history = "\n".join(
        [f"User: {m.content}" if m.type == "human" else f"Assistant: {m.content}" 
         for m in memory.chat_memory.messages[-6:]]
    )
    
    prompt = format_prompt(context, question)
    answer = generate_answer_ollama(prompt)

    # Update memory
    memory.chat_memory.add_user_message(question)
    memory.chat_memory.add_ai_message(answer)

    # Evaluate
    relevance_score, groundedness_score = evaluate_rag_retrieval(
        question, relevant_chunks, answer, embedder
    )

    print(f"\n🔹 প্রশ্ন {i}: {question}")
    print(f"✅ উত্তর: {answer}")
    print(f"📊 RAG Evaluation:")
    print(f"   🔸 Relevance Score: {relevance_score:.4f}")
    print(f"   🔸 Groundedness Score: {groundedness_score:.4f}")
    print("-" * 90)


🔹 প্রশ্ন 1: অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?
✅ উত্তর: অনুপমের ভাষায় "অন্নপূর্ণ" শব্দে সুপুরুষ কাকে বলা হয়েছে। এটি গজাননের ( কার্তিকের ( প্রজাপতির অন্নপূর্ণার ৭০ সুপুরুষ বটে হয়।
📊 RAG Evaluation:
   🔸 Relevance Score: 0.8340
   🔸 Groundedness Score: 0.8370
------------------------------------------------------------------------------------------

🔹 প্রশ্ন 2: অনুপম কাকে ভাগ্যদেবতা বলেছেন?
✅ উত্তর: মামাকে ভাগ্যদেবতা বলেছেন।
📊 RAG Evaluation:
   🔸 Relevance Score: 0.8566
   🔸 Groundedness Score: 0.7707
------------------------------------------------------------------------------------------

🔹 প্রশ্ন 3: বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?
✅ উত্তর: কল্যাণীর বিয়ের সময় থেকে পূর্বে ১৪ বছর ছিল।
📊 RAG Evaluation:
   🔸 Relevance Score: 0.8561
   🔸 Groundedness Score: 0.8178
------------------------------------------------------------------------------------------

🔹 প্রশ্ন 4: অনুপমের জীবনে মামার ভূমিকা কী ছিল?
✅ উত্তর: অনুপমের জীবিকায় মামার ভূমিকা 'ভাগ্যদেবতার প্রধান এজেন্ট' ছ